In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, dayofmonth, month, quarter, year, dayofweek, date_format, sum as spark_sum

# Set environment vars to load jars
os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--jars jars/hadoop-azure-3.3.6.jar," # Hadoop connector (wasb + wasbs)
    "jars/mysql-connector-j-9.3.0.jar," # Allow write to mysql DB
    "jars/azure-storage-8.6.6.jar," # Azure SDK for Java (allow communication between Hadoop, Spark with Blobs Storage)
    "jars/jetty-client-9.4.43.v20210629.jar," # I don't know...
    "jars/jetty-http-9.4.43.v20210629.jar," ###############
    "jars/jetty-io-9.4.43.v20210629.jar," #################
    "jars/jetty-util-9.4.43.v20210629.jar," ################
    "jars/jetty-util-ajax-9.4.43.v20210629.jar " ############
    "pyspark-shell"
)


In [2]:
# Init spark session
spark = SparkSession.builder \
    .appName("DW data load") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/09 08:41:59 WARN Utils: Your hostname, lenovo-slim, resolves to a loopback address: 127.0.1.1; using 192.168.199.13 instead (on interface wlp2s0)
25/07/09 08:41:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/07/09 08:42:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/09 08:42:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
mysql_url = "jdbc:mysql://localhost:3306/store_dw"
mysql_props = {
    "user": "bnguyen",
    "password": ".Tldccmcbtldck2",
    "driver": "com.mysql.cj.jdbc.Driver"
}

In [4]:
# Silver access key
spark.conf.set(
    "fs.azure.account.key.mysilver.blob.core.windows.net",
    "bAthp0pVBfqEtyCvJElSX7MeI7ejSLa6cjuPoMz0Gg/69uzEW01y4URMDXsdFCrkpc9M54cDHnXs+AStj1gExQ=="
)

# Gold
spark.conf.set(
    "fs.azure.account.key.mygold.dfs.core.windows.net",
    "wRPXTwWCVxWwUpavEh62A5wzLdUvRTGeB3tZKP3eRbig7ca8ZN51l0kWS32kcbH/ddQ/jNXBzqDC+AStOzXlyw=="
)

In [5]:
df = spark.read.parquet("wasbs://silver@mysilver.blob.core.windows.net/Products")
df.show()

25/07/09 08:42:45 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-azure-file-system.properties,hadoop-metrics2.properties


+---------+--------------------+--------------------+-----+----------+--------+-------------------+-------------------+
|ProductID|                Name|         Description|Price|CategoryID|SellerID|          CreatedAt|          UpdatedAt|
+---------+--------------------+--------------------+-----+----------+--------+-------------------+-------------------+
|        1|         Lamp AS-586|A decorative lamp...|  402|         3|    2320|2025-01-14 21:04:24|2025-01-19 19:49:46|
|        2|       Tablet sp-018|A lightweight tab...|  378|         1|    3708|2023-11-14 09:02:40|2025-05-07 04:33:14|
|        3|     Lipstick CB-813|A long-lasting li...|   63|         5|    3100|2024-04-30 17:20:16|2024-10-14 09:24:58|
|        4|     Notebook jo-312|A ruled notebook ...|  147|        10|    2919|2024-12-10 10:14:21|2025-04-15 17:06:53|
|        5|      T-Shirt oh-467|A comfortable cot...|   94|         2|     692|2025-01-24 01:21:32|2024-08-08 15:03:21|
|        6|     Lipstick nH-281|A long-l

In [19]:
# 1. DimProduct
products = spark.read.parquet("wasbs://silver@mysilver.blob.core.windows.net/Products")
dim_product = products.select(
    col("ProductID"),
    col("Name").alias("ProductName"),
    col("CategoryID"),
    col("SellerID")
)
dim_product.write.jdbc(mysql_url, "DimProduct", mode="append", properties=mysql_props)
dim_product.write.mode("overwrite").parquet("abfss://gold-test@mygold.dfs.core.windows.net/DimProduct")

In [20]:
# 2. DimCategory
categories = spark.read.parquet("wasbs://silver@mysilver.blob.core.windows.net/ProductCategories")
dim_category = categories.select(
    col("CategoryID"),
    col("CategoryName")
)
dim_category.write.jdbc(mysql_url, "DimCategory", mode="append", properties=mysql_props)
dim_category.write.mode("overwrite").parquet("abfss://gold-test@mygold.dfs.core.windows.net/DimCategory")

In [21]:
# 3. DimSeller
sellers = spark.read.parquet("wasbs://silver@mysilver.blob.core.windows.net/Sellers")
dim_seller = sellers.select(
    col("SellerID"),
    col("Name").alias("SellerName")
)
dim_seller.write.jdbc(mysql_url, "DimSeller", mode="append", properties=mysql_props)
dim_seller.write.mode("overwrite").parquet("abfss://gold-test@mygold.dfs.core.windows.net/DimSeller")

In [22]:
# 4. DimCustomer
customers = spark.read.parquet("wasbs://silver@mysilver.blob.core.windows.net/Customers")
dim_customer = customers.select(
    col("CustomerID"),
    col("Name").alias("CustomerName")
)
dim_customer.write.jdbc(mysql_url, "DimCustomer", mode="append", properties=mysql_props)
dim_customer.write.mode("overwrite").parquet("abfss://gold-test@mygold.dfs.core.windows.net/DimCustomer")

In [23]:
# 5. DimOrderStatus
order_status = spark.read.parquet("wasbs://silver@mysilver.blob.core.windows.net/OrderStatus")
dim_order_status = order_status.select(
    col("StatusID"),
    col("StatusName")
)
dim_order_status.write.jdbc(mysql_url, "DimOrderStatus", mode="append", properties=mysql_props)
dim_order_status.write.mode("overwrite").parquet("abfss://gold-test@mygold.dfs.core.windows.net/DimOrderStatus")

In [24]:
# 6. DimDate (from Orders)
orders = spark.read.parquet("wasbs://silver@mysilver.blob.core.windows.net/Orders")
dim_date = orders.select(
    date_format(col("CreatedAt"), "yyyyMMdd").cast("int").alias("DateKey"),
    col("CreatedAt").cast("date").alias("Date"),
    dayofmonth(col("CreatedAt")).alias("Day"),
    month(col("CreatedAt")).alias("Month"),
    quarter(col("CreatedAt")).alias("Quarter"),
    year(col("CreatedAt")).alias("Year"),
    dayofweek(col("CreatedAt")).alias("DayOfWeek")
).distinct()
dim_date.write.jdbc(mysql_url, "DimDate", mode="append", properties=mysql_props)
dim_date.write.mode("overwrite").parquet("abfss://gold-test@mygold.dfs.core.windows.net/DimDate")

In [25]:
# 7. DimReason
reasons = spark.read.parquet("wasbs://silver@mysilver.blob.core.windows.net/Reasons")
dim_reason = reasons.select(
    col("ReasonID"),
    col("ReasonType"),
    col("ReasonDescription")
)
dim_reason.write.jdbc(mysql_url, "DimReason", mode="append", properties=mysql_props)
dim_reason.write.mode("overwrite").parquet("abfss://gold-test@mygold.dfs.core.windows.net/DimReason")

In [26]:
# Fact sales
order_items = spark.read.parquet("wasbs://silver@mysilver.blob.core.windows.net/OrderItems").alias("oi")
orders = spark.read.parquet("wasbs://silver@mysilver.blob.core.windows.net/Orders").alias("o")
payments = spark.read.parquet("wasbs://silver@mysilver.blob.core.windows.net/Payments").alias("p")
products = spark.read.parquet("wasbs://silver@mysilver.blob.core.windows.net/Products").alias("pr")

# Only consider orders that have a payment
paid_orders = payments.select("OrderID").distinct().alias("po")

# Join OrderItems with Orders and filter for paid orders
fact_sales = (
    order_items
    .join(orders, col("oi.OrderID") == col("o.OrderID"))
    .join(paid_orders, col("oi.OrderID") == col("po.OrderID"), "inner")
    .join(products, col("oi.ProductID") == col("pr.ProductID"))
    .join(payments, col("oi.OrderID") == col("p.OrderID"), "inner")  # Join with payments
    .select(
        col("oi.OrderItemID"),
        col("oi.OrderID"),
        col("oi.ProductID"),
        col("pr.SellerID"),
        col("o.CustomerID"),
        col("pr.CategoryID"),
        date_format(col("o.CreatedAt"), "yyyyMMdd").cast("int").alias("OrderDateKey"),
        col("o.StatusID"),
        col("oi.Quantity").cast("int").alias("Quantity"),
        col("oi.CurrentPrice").cast("double").alias("CurrentPrice"),
        (col("oi.Quantity").cast("int") * col("oi.CurrentPrice").cast("double")).alias("Revenue"),
        col("p.CreatedAt").alias("CreatedAt")  # Use payment time
    )
)
fact_sales.write.jdbc(mysql_url, "FactSales", mode="append", properties=mysql_props)
fact_sales.write.mode("overwrite").parquet("abfss://gold-test@mygold.dfs.core.windows.net/FactSales")

In [27]:
# Fact reasons
reasons = spark.read.parquet("wasbs://silver@mysilver.blob.core.windows.net/Reasons")
orders = spark.read.parquet("wasbs://silver@mysilver.blob.core.windows.net/Orders")

# Join Reasons with Orders to get OrderDateKey and StatusID
fact_order_reason = (
    reasons
    .join(orders, reasons.OrderID == orders.OrderID, "inner")
    .select(
        reasons.ReasonID,
        reasons.OrderID,
        date_format(orders.CreatedAt, "yyyyMMdd").cast("int").alias("OrderDateKey"),
        orders.StatusID,
        reasons.CreatedAt.alias("CreatedAt")  # Use Reasons.CreatedAt
    )
)
fact_order_reason.write.jdbc(mysql_url, "FactOrderReason", mode="append", properties=mysql_props)
fact_order_reason.write.mode("overwrite").parquet("abfss://gold-test@mygold.dfs.core.windows.net/FactOrderReason")